In [1]:
# Useful starting lines
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2

from implementation import *
from utilities import *

# Load and clean the data

In [2]:
y,x,ids=load_csv_data("train.csv")
y_test,x_test,ids_test=load_csv_data("test.csv")

In [22]:
# tej colonnes où y'a des -999

x[x==-999]=np.NaN
z=x[:,~np.isnan(x).any(axis=0)] #z is our matrix of observations with size (number of observations, number of features without -999)

#z=one_hot_jet_num(z,17)
#z=build_poly_with_one_hot(z,5,17,True,3)

#z2=interaction_prod(z,0,False)
#z1 = build_poly(z,3)
#z3=build_poly(z2,3)
#z = np.c_[z1,z2]
#z=np.c_[z1,z3]
#add_ones(z)
#np.column_stack((z, np.ones(z.shape[0])))
#np.column_stack((z, np.ones(z.shape[0])))
print(z.shape)

(250000, 19)


In [ ]:
x_test[x_test==-999]=np.NaN
z_test=x_test[:,~np.isnan(x_test).any(axis=0)] #z is our matrix of observations with size (number of observations, number of features without -999)
z1_test = build_poly(z_test,9)
z2_test=interaction_prodbis(z_test,0,False)
z3_test=build_poly(z2_test,6)
z_test = np.c_[z1_test,z3_test]
#z_test=np.c_[z1_test,z3_test]

In [23]:
# cross validation for 2 --> TO DO: FOR K

Strain1_z, Stest1_z, Strain1_y, Stest1_y = split_data(z,y,0.8)

# standardize the data (only centered here)

#Strain1_z = standardize(Strain1_z)
#Stest1_z = standardize(Stest1_z)
#Strain1_y = standardize(Strain1_y)
#Stest_y = standardize(Stest1_y)

# Calculate MSE for result from least squares 

In [72]:
beta1,l = least_squares(Strain1_y, Strain1_z)      #beta from y = X^T*beta --> missing the column of ones'
#print(beta1)
res1 = definitive_res(Stest1_z.dot(beta1))
#print(res1)
len(Stest1_y[Stest1_y==res1])/len(Stest1_y)

0.7231

# Calculate MSE for results from least squares GD and SGD

In [67]:
initial_w = np.zeros(Strain1_z.shape[1])    #w is an array of size = number of features (i.e. number of columns in z)
max_iters = 5000
gamma = 0.000001

# GD

In [70]:
beta2 ,l= least_squares_GD(Strain1_y, Strain1_z, initial_w, max_iters, gamma)      #beta from y = X*beta --> missing the column of ones'
res2 = definitive_res(Stest1_z.dot(beta2))
len(Stest1_y[Stest1_y==res2])/len(Stest1_y)

KeyboardInterrupt: 

# SGD

In [69]:
batch_size = 1

beta3 = least_squares_SGD(Strain1_y, Strain1_z, initial_w, batch_size, max_iters, gamma)      #beta from y = X*beta --> missing the column of ones'
res3 = definitive_res(Stest1_z.dot(beta3))
len(Stest1_y[Stest1_y==res3])/len(Stest1_y)

0.7179

# Calculate other LS for Ridge Regression 

In [ ]:
lambdas = np.logspace(-5, 0, 15)

# bluid poly
tx_tr = build_poly(Strain1_z, 1)
tx_te = build_poly(Stest1_z, 1)

# ridge regression with different lambda
rmse_tr = []
rmse_te = []
for ind, lambda_ in enumerate(lambdas):
    # ridge regression
    weight = ridge_regression(Strain1_y, tx_tr, lambda_)
    rmse_tr.append(np.sqrt(2 * compute_mse(Strain1_y, tx_tr, weight)))
    rmse_te.append(np.sqrt(2 * compute_mse(Stest1_y, tx_te, weight)))
print(rmse_tr, rmse_te)
#print("last weight:",weight)

In [ ]:
lmin=-10
lmax=0
n=10
y=Strain1_y
x=Strain1_z
y_te=Stest1_y
x_te=Stest1_z
ratio=[]
ratios=[]
weights=[]
lambdas = np.logspace(lmin, lmax, n)
for ind, lambda_ in enumerate(lambdas):
    weight = ridge_regression(y, x, lambda_)
    ratio.append(ratio_prediction_threshold(y_te,x_te,weight,-0.1))
    ratios.append(ratio_prediction_threshold(y_te,x_te,weight,0.1))
    weights.append(weight)
    print(ind)
indx1=np.argmax(ratio)
indx2=np.argmax(ratios)
weight=weights[indx]
lambdaa=lambdas[indx]


In [ ]:
print(np.max(ratio),indx)
print(np.max(ratios),indx)

In [ ]:
pred=definitive_res(z_test.dot(weight))
create_csv_submission(ids_test,pred,"testsubs")

In [73]:
beta4,losss=logistic_regression(Strain1_y, Strain1_z, initial_w, max_iters, gamma)
res4 = definitive_res(Stest1_z.dot(beta4))
len(Stest1_y[Stest1_y==res4])/len(Stest1_y)

0.6554

In [83]:
Stest1_z.dot(beta4)

array([-13890454.92794211, -23555887.34881421, -17856747.65964799, ...,
       -29212526.47376706, -21439167.74731878,  -8626981.51692929])